In [ ]:
"""
Script to run experimental comparisons of all algorithms
"""

import os
import sys
import time
import csv
sys.path.append(os.path.join(os.path.dirname(__file__), 'src'))

from environment import GridEnvironment
from agent import DeliveryAgent

def run_experiments():
    """Run comprehensive experiments on all maps"""

    maps = ['small.map', 'medium.map', 'large.map', 'dynamic.map']
    algorithms = ['bfs', 'ucs', 'astar', 'hillclimb', 'annealing']

    results = []

    for map_name in maps:
        print(f"\n=== Testing on {map_name} ===")

        env = GridEnvironment(0, 0)
        env.load_from_file(f'maps/{map_name}')

        goal = env.delivery_points[0] if env.delivery_points else (env.width-1, env.height-1)

        for algorithm in algorithms:
            agent = DeliveryAgent(env)

            start_time = time.time()
            result = agent.plan_path(algorithm, env.agent_start, goal)
            end_time = time.time()

            result['algorithm'] = algorithm
            result['map'] = map_name
            result['total_time'] = end_time - start_time

            results.append(result)

            status = "✓" if result['success'] else "✗"
            print(f"{status} {algorithm:<10} | Cost: {result['cost']:>6} | "
                  f"Nodes: {result['nodes_expanded']:>6} | Time: {result['total_time']:>7.4f}s")

    # Save results to CSV
    with open('results/experimental_results.csv', 'w', newline='') as csvfile:
        fieldnames = ['algorithm', 'map', 'success', 'cost', 'nodes_expanded', 'total_time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for result in results:
            writer.writerow({
                'algorithm': result['algorithm'],
                'map': result['map'],
                'success': result['success'],
                'cost': result['cost'],
                'nodes_expanded': result['nodes_expanded'],
                'total_time': result['total_time']
            })

    print("\n=== Results saved to results/experimental_results.csv ===")
    return results

if __name__ == '__main__':
    run_experiments()